In [1]:
import numpy as np

In [2]:
# Puzzle input
file = 'input.txt'
board_width = 150

In [3]:
file = open(file, 'r')
input = file.readlines()
file.close()

In [4]:
import sys

def print_board(command=None):

    original_stdout = sys.stdout # Save a reference to the original standard output

    with open('board_progress.txt', 'a') as f:
        sys.stdout = f # Change the standard output to the file we created.
        
        if command != None:
            print('Command: '+str(command))
        
        for i in range(len(board)):
            line = ""
            for j in range(board_width):
                if board[i,j] == 0:
                    line += " "
                elif board[i,j] == 1:
                    line += "."
                elif board[i,j] == 2:
                    line += "#"
                elif board[i,j] == 10:
                    line += ">"
                elif board[i,j] == 11:
                    line += "v"
                elif board[i,j] == 12:
                    line += "<"
                elif board[i,j] == 13:
                    line += "^"
            print(line)

        sys.stdout = original_stdout # Reset the standard output to its original value

In [5]:
# build grid
board = np.zeros((len(input)-2, board_width))
for i,line in enumerate(input[:-2]):
    for j,c in enumerate(line):
        if c == ".":
            board[i,j] = 1
        elif c == "#":
            board[i,j] = 2
#print_board()

In [6]:
# set starting point
current_ori = 10
current_pos = (0,50)
board[0,50] = 10   

In [7]:
# parse path
path = [] 
number = ""
for c in input[-1].strip():
    if c == 'R' or c == 'L':
        path.append(int(number))
        number = ""
        path.append(c)
    else:
        number += c
path.append(int(number))

In [8]:
def find_new_target_pos(source_pos):
    new_pos = source_pos
    new_ori = 10
    if source_pos[0] == 0 and source_pos[1] >= 50 and source_pos[1] < 100:
        new_pos = (source_pos[1]+100,0)
        new_ori = 10
        #checked
    elif source_pos[1] == 0 and source_pos[0] >= 150:
        new_pos = (0,source_pos[0]-100)
        new_ori = 11
        #checked
    elif source_pos[0] == 0 and source_pos[1] >= 100:
        new_pos = (199,source_pos[1]-100)
        new_ori = 13
    elif source_pos[0] == 199:
        new_pos = (0,source_pos[1]+100)
        new_ori = 11
    elif source_pos[0] < 50 and source_pos[1] == 50:
        new_pos = (149-source_pos[0],0)
        new_ori = 10
        # checked
    elif source_pos[1] == 0 and source_pos[0] < 150:
        new_pos = (149-source_pos[0],50)
        new_ori = 10
        # checked
    elif source_pos[1] == 149:
        new_pos = (149-source_pos[0],99)
        new_ori = 12
        # checked
    elif source_pos[1] == 99 and source_pos[0] > 99:
        new_pos = (149-source_pos[0],149)
        new_ori = 12
        # checked
    elif source_pos[0] == 49:
        new_pos = (source_pos[1]-50,99)
        new_ori = 12
    elif source_pos[1] == 99 and source_pos[0] <= 99:
        new_pos = (49,source_pos[0]+50)
        new_ori = 13
    elif source_pos[1] == 50 and source_pos[0] > 49:
        new_pos = (100,source_pos[0]-50)
        new_ori = 11
    elif source_pos[0] == 100:
        new_pos = (source_pos[1]+50,50)
        new_ori = 10
    elif source_pos[0] == 149:
        new_pos = (source_pos[1]+100,49)
        new_ori = 12
        # checked
    elif source_pos[1] == 49:
        new_pos = (149,source_pos[0]-100)
        new_ori = 13

    if board[new_pos] in [1,10,11,12,13]:
        return new_pos, new_ori
    elif new_pos == 2:
        return None, None

In [9]:
def go_single_step(source_pos, target_pos, ori):
    if target_pos[0] < 0 or target_pos[0] >= len(board) or target_pos[1] < 0 or target_pos[1] >= board_width:
        return find_new_target_pos(source_pos)
    elif board[target_pos] in [1,10,11,12,13]:
        return target_pos, ori
    elif board[target_pos] == 2:
        return None, None
    elif board[target_pos] == 0:
        return find_new_target_pos(source_pos)
    return None, None

In [10]:
def go_x_steps(x, ori, pos):
    new_pos = pos
    new_ori = ori
    for i in range(x):
        if ori == 10:
            pos, ori = go_single_step(pos, (pos[0],pos[1]+1), ori) or (None,None)
        elif ori == 11:
            pos, ori = go_single_step(pos, (pos[0]+1,pos[1]), ori) or (None,None)
        elif ori == 12:
            pos, ori = go_single_step(pos, (pos[0],pos[1]-1), ori) or (None,None)
        elif ori == 13:
            pos, ori = go_single_step(pos, (pos[0]-1,pos[1]), ori) or (None,None)
            
        if pos == None:
            break
        else:
            new_pos = pos
            new_ori = ori
            board[new_pos] = ori
    return new_pos, new_ori

In [11]:
def turn(c, ori):
    if c == 'R':
        ori += 1
    elif c == 'L':
        ori -= 1
    if ori == 14:
        ori = 10
    if ori == 9:
        ori = 13
    board[current_pos] = ori 
    return ori     

In [12]:
for c in path:
    if type(c) == int:
        current_pos, current_ori = go_x_steps(c, current_ori, current_pos)
    else:
        current_ori = turn(c, current_ori)

In [13]:
def compute_result(current_pos, current_ori):
    result = 1000 * (current_pos[0] + 1)
    result += 4 * (current_pos[1] + 1)
    result += current_ori - 10
    return result

In [14]:
compute_result(current_pos, current_ori)

126017